In [2]:
from google.cloud import bigquery
from google.cloud import aiplatform
import bigframes.pandas as bpd
import pandas as pd
from vertexai.language_models._language_models import TextGenerationModel
from bigframes.ml.cluster import KMeans
from bigframes.ml.model_selection import train_test_split

In [3]:
project_id = 'qwiklabs-gcp-02-b016a44b746f'
dataset_name = "ecommerce"
model_name = "customer_segmentation_model"
table_name = "customer_stats"
location = "us-central1"
client = bigquery.Client(project=project_id)
aiplatform.init(project=project_id, location=location)

In [4]:
%%bigquery
CREATE OR REPLACE TABLE ecommerce.customer_stats AS
SELECT
  user_id,
  DATE_DIFF(CURRENT_DATE(), CAST(MAX(order_created_date) AS DATE), day) AS days_since_last_order, ---RECENCY
  COUNT(order_id) AS count_orders, --FREQUENCY
  AVG(sale_price) AS average_spend --MONETARY
  FROM (
      SELECT
        user_id,
        order_id,
        sale_price,
        created_at AS order_created_date
        FROM `bigquery-public-data.thelook_ecommerce.order_items`
        WHERE
        created_at
            BETWEEN '2022-01-01' AND '2023-01-01'
  )
GROUP BY user_id;

Query is running:   0%|          |

""


In [8]:
customer_stats_df = bpd.read_gbq(f'{project_id}.{dataset_name}.{table_name}')
customer_stats_df.head(10).to_pandas()

,user_id,days_since_last_order,count_orders,average_spend
0,53325,645,1,68.0
1,86692,663,1,221.979996
2,11179,731,1,51.810001
3,55446,743,1,56.0
4,10038,896,1,20.0
5,10417,797,1,62.0
6,74386,731,4,141.669999
7,56127,719,2,50.75
8,42097,727,1,37.900002
9,80439,783,3,60.543333


In [9]:
# prompt: 1. Split df (using random state and test size 0.2) into test and training data for a K-means clustering algorithm store these as df_test and df_train. 2. Create a K-means cluster model using bigframes.ml.cluster KMeans with 5 clusters. 3. Save the model using the to_gbq method where the model name is project_id.dataset_name.model_name.

df_train, df_test = train_test_split(customer_stats_df, test_size=0.2, random_state=42)
kmeans = KMeans(n_clusters=5)
kmeans.fit(df_train)
kmeans.to_gbq(f'{project_id}.{dataset_name}.{model_name}')

KMeans(distance_type='EUCLIDEAN', init='KMEANS_PLUS_PLUS', n_clusters=5)

In [13]:
# prompt: 1. Call the K-means prediction model on the df dataframe, and store the results as predictions_df and show the first 10 records.

predictions_df = kmeans.predict(df_test)
predictions_df.head(10)

,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,user_id,days_since_last_order,count_orders,average_spend
3,3,"[{'CENTROID_ID': 3, 'DISTANCE': 0.820775583359...",55446,743,1,56.0
10,5,"[{'CENTROID_ID': 5, 'DISTANCE': 1.987621043383...",49502,962,3,39.643332
23,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.576457682852...",30291,752,1,54.0
24,5,"[{'CENTROID_ID': 5, 'DISTANCE': 0.983656962237...",27039,831,2,34.995
27,1,"[{'CENTROID_ID': 1, 'DISTANCE': 1.280905232278...",427,769,1,98.0
30,1,"[{'CENTROID_ID': 1, 'DISTANCE': 1.017038723399...",38874,638,1,81.949997
31,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.981412929933...",4139,669,1,24.99
50,3,"[{'CENTROID_ID': 3, 'DISTANCE': 0.923105357427...",83500,759,2,67.03
56,3,"[{'CENTROID_ID': 3, 'DISTANCE': 0.382556305256...",73103,721,1,50.0
57,4,"[{'CENTROID_ID': 4, 'DISTANCE': 0.933986090418...",84558,832,1,19.950001


In [14]:
#prompt: 1. Using predictions_df, and matplotlib, generate a scatterplot. 2. On the x-axis of the scatterplot, display days_since_last_order and on the y-axis, display average_spend from predictions_df. 3. Color by cluster. 4. The chart should be titled "Attribute grouped by K-means cluster."

import matplotlib.pyplot as plt

# Create the scatter plot
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plt.scatter(predictions_df['days_since_last_order'], predictions_df['average_spend'], c=predictions_df['cluster'], cmap='viridis')

# Customize the plot
plt.title('Attribute grouped by K-means cluster')
plt.xlabel('Days Since Last Order')
plt.ylabel('Average Spend')
plt.colorbar(label='Cluster ID')

# Display the plot
plt.show()

KeyError: 'cluster'

<Figure size 1000x600 with 0 Axes>

In [15]:
query = """
SELECT
 CONCAT('cluster ', CAST(centroid_id as STRING)) as centroid,
 average_spend,
 count_orders,
 days_since_last_order
FROM (
 SELECT centroid_id, feature, ROUND(numerical_value, 2) as value
 FROM ML.CENTROIDS(MODEL `{0}.{1}`)
)
PIVOT (
 SUM(value)
 FOR feature IN ('average_spend',  'count_orders', 'days_since_last_order')
)
ORDER BY centroid_id
""".format(dataset_name, model_name)

df_centroid = client.query(query).to_dataframe()
df_centroid.head()

,centroid,average_spend,count_orders,days_since_last_order
0,cluster 1,55.14,1.32,707.55
1,cluster 2,71.79,3.76,754.56
2,cluster 3,58.54,1.31,703.01
3,cluster 4,57.28,1.30,886.03
4,cluster 5,61.14,1.33,891.34


In [16]:
df_query = client.query(query).to_dataframe()
df_query.to_string(header=False, index=False)

cluster_info = []
for i, row in df_query.iterrows():
 cluster_info.append("{0}, average spend ${2}, count of orders per person {1}, days since last order {3}"
  .format(row["centroid"], row["count_orders"], row["average_spend"], row["days_since_last_order"]) )

cluster_info = (str.join("\n", cluster_info))
print(cluster_info)

cluster 1, average spend $55.14, count of orders per person 1.32, days since last order 707.55
cluster 2, average spend $71.79, count of orders per person 3.76, days since last order 754.56
cluster 3, average spend $58.54, count of orders per person 1.31, days since last order 703.01
cluster 4, average spend $57.28, count of orders per person 1.3, days since last order 886.03
cluster 5, average spend $61.14, count of orders per person 1.33, days since last order 891.34


In [17]:
prompt = f"""
You're a creative brand strategist, given the following clusters, come up with \
creative brand persona, a catchy title, and next marketing action, \
explained step by step.

Clusters:
{cluster_info}

For each Cluster:
* Title:
* Persona:
* Next marketing step:
"""

In [18]:
# prompt: Use the Vertex AI language_models API to call the PaLM2 text-bison model and generate a marketing campaign using the variable prompt. Use the following model settings: max_output_tokens=1024, temperature=0.4

model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict(prompt, max_output_tokens=1024, temperature=0.4)
print(response.text)

**Cluster 1**

* Title: The Loyal Customers
* Persona: These are your most loyal customers. They have been shopping with you for a long time and they spend a lot of money. They are the ones who are most likely to refer you to their friends and family.
* Next marketing step: Send them a personalized email or letter thanking them for their loyalty. Offer them a special discount or promotion.

**Cluster 2**

* Title: The Big Spenders
* Persona: These are your customers who spend the most money. They are typically high-income individuals who are looking for the best quality products and services. They are also the most likely to be brand loyal.
* Next marketing step: Host an exclusive event for your big spenders. Invite them to a VIP shopping experience or a special event.

**Cluster 3**

* Title: The Frequent Buyers
* Persona: These are your customers who buy from you on a regular basis. They are typically looking for convenience and value. They are also the most likely to be repeat custo